In [1]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [2]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2022-04-21 11:59:22--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2022-04-21 11:59:22--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?RLsaOpDPPfMn8NX-xlYvxpOr7qNwGDpiCRqgIdR6_TqtSBnc48T9VcYa3XW41jqqT-RH0DksxFHfLu-OD-2StxNlPe0fidPuyIQsML36XY9Ah

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [4]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-gb_shwdj
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-gb_shwdj
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=2534b41f921d19a416901d3e551047d06da35855c9cc11404fc13e526dab8344
  Stored in directory: /tmp/pip-ephem-wheel-cache-6ejkd4ok/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [52]:
%%writefile CudaLodepng1.cu

#include <stdio.h>
#include <stdlib.h>

#include "/content/lodepng.h"

//each pixel has 4 values:
//Red (R)
//Green (G)
//Blue (B)
//Transparency (T)

//negative filter (reversing the RGB values)
//0-255   max-val    255-40

__global__ void AverageFilter(unsigned char * inputImage, unsigned char * outputImage, int s, int w){

int ravg;
int gavg;
int bavg;
int t;
int width = w; //gets the width of the image

int threadIndex = blockDim.x * blockIdx.x + threadIdx.x;

int pixel = threadIndex*4;

//printf("uid = %d\n", size);
int size = s; // gets the size of the image

if(pixel < size)
{
  printf("uid = %d\n", pixel);

  t = inputImage[pixel+3];

  //Average Filtering

  ravg = (inputImage[pixel] + inputImage[pixel - 4] + inputImage[pixel + 4] +
          inputImage[pixel + width*4] + inputImage[pixel + width*4 - 4] + inputImage[pixel + width*4 + 4] +
          inputImage[pixel - width*4] + inputImage[pixel - width*4 - 4] + inputImage[pixel - width*4 + 4])/9;

  gavg = (inputImage[pixel + 1] + inputImage[pixel + 1 - 4] + inputImage[pixel + 1 + 4] +
          inputImage[pixel + 1 + width*4] + inputImage[pixel + 1 + width*4 - 4] + inputImage[pixel + 1 + width*4 + 4] +
          inputImage[pixel + 1 - width*4] + inputImage[pixel + 1 - width*4 - 4] + inputImage[pixel + 1 - width*4 + 4])/9;            

  bavg = (inputImage[pixel + 2] + inputImage[pixel + 2 - 4] + inputImage[pixel + 2 + 4] +
          inputImage[pixel + 2 + width*4] + inputImage[pixel + 2 + width*4 - 4] + inputImage[pixel + 2 + width*4 + 4] +
          inputImage[pixel + 2 - width*4] + inputImage[pixel + 2 - width*4 - 4] + inputImage[pixel + 2 - width*4 + 4])/9;  

  outputImage[pixel] = ravg;
  outputImage[pixel+1] = gavg;
  outputImage[pixel+2] = bavg;
  outputImage[pixel+3] = t;       
}

}

int main(int argc, char ** argv){

  unsigned int errorDecode; //variable will hold whether there was an issue with loading in the png file
  unsigned char* cpuImage; //this variable will hold all of our image data
  unsigned int width, height; //holds the width and height of image
  
  char * filename = argv[1];
  char * newFilename = argv[2];

  errorDecode = lodepng_decode32_file(&cpuImage, &width, &height, filename); // (where to store the image data, width, height, which file?)
  if(errorDecode){
    printf("error %u: %s\n", errorDecode, lodepng_error_text(errorDecode));
  }
  
  int arraySize = width*height*4;
  int memorySize = arraySize * sizeof(unsigned char);
  
  printf(" w %d, h %d, s %d", width, height, arraySize);

  unsigned char cpuOutImage[arraySize];
  
  unsigned char* gpuInput;
  unsigned char* gpuOutput;
  unsigned int * gpuSize;
  
  cudaMalloc( (void**) &gpuInput, memorySize);
  cudaMalloc( (void**) &gpuOutput, memorySize);
 // cudaMalloc( (void**) &gpuSize, sizeof(unsigned int));

  cudaMemcpy(gpuInput, cpuImage, memorySize, cudaMemcpyHostToDevice);
 // cudaMemcpy(gpuSize, (unsigned int) memorySize, sizeof(unsigned int), cudaMemcpyHostToDevice);

  AverageFilter<<< 1000, 1000 >>>(gpuInput, gpuOutput, arraySize, width);
  cudaDeviceSynchronize();

  cudaMemcpy(cpuOutImage, gpuOutput, memorySize, cudaMemcpyDeviceToHost);
  
  unsigned int errorEncode = lodepng_encode32_file(newFilename, cpuOutImage, width, height);
  if(errorEncode) {
  printf("error %u: %s\n", errorEncode, lodepng_error_text(errorEncode));
  }

  //free(image);

}

Writing CudaLodepng1.cu


In [53]:
%%shell

nvcc CudaLodepng1.cu lodepng.cpp -o output

./output lena.png lena_avg.png


Streaming output truncated to the last 5000 lines.
uid = 956288
uid = 956292
uid = 956296
uid = 956300
uid = 956304
uid = 956308
uid = 956312
uid = 956316
uid = 956320
uid = 956324
uid = 956328
uid = 956332
uid = 956336
uid = 956340
uid = 956344
uid = 956348
uid = 956352
uid = 956356
uid = 956360
uid = 956364
uid = 956368
uid = 956372
uid = 956376
uid = 956380
uid = 959200
uid = 959204
uid = 959208
uid = 959212
uid = 959216
uid = 959220
uid = 959224
uid = 959228
uid = 959232
uid = 959236
uid = 959240
uid = 959244
uid = 959248
uid = 959252
uid = 959256
uid = 959260
uid = 959264
uid = 959268
uid = 959272
uid = 959276
uid = 959280
uid = 959284
uid = 959288
uid = 959292
uid = 959296
uid = 959300
uid = 959304
uid = 959308
uid = 959312
uid = 959316
uid = 959320
uid = 959324
uid = 957024
uid = 957028
uid = 957032
uid = 957036
uid = 957040
uid = 957044
uid = 957048
uid = 957052
uid = 957056
uid = 957060
uid = 957064
uid = 957068
uid = 957072
uid = 957076
uid = 957080
uid = 957084
uid = 957088
